**1 - data preprocessing**

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('df_clean.csv',index_col=0).reset_index()

In [3]:
df.drop(labels=['day','month','index','pdays'],axis=1,inplace=True)
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,cellular,261,1,0,failure,no
1,44,technician,single,secondary,no,29,yes,no,cellular,151,1,0,failure,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,cellular,76,1,0,failure,no
3,35,management,married,tertiary,no,231,yes,no,cellular,139,1,0,other,no
4,28,management,single,tertiary,no,447,yes,yes,cellular,217,1,0,failure,no


In [4]:
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
duration      int64
campaign      int64
previous      int64
poutcome     object
y            object
dtype: object

In [5]:
#change categorical values into numerical values
df = df.replace('no',0)
df = df.replace('yes',1)

df = df.replace('cellular',0)
df = df.replace('telephone',1)

df = df.replace('success',1)
df = df.replace('failure',0)
df = df.replace('other',0)

In [6]:
for i in range(len(df)):
    if df.loc[i,'job'] == 'management':
        df.loc[i,'job'] = 1
    else:
        df.loc[i,'job'] = 0

In [7]:
for i in range(len(df)):
    if df.loc[i,'marital'] == 'married':
        df.loc[i,'marital'] = 2
    elif df.loc[i,'marital'] == 'single':
        df.loc[i,'marital'] = 1
    else:
        df.loc[i,'marital'] = 0

In [8]:
for i in range(len(df)):
    if df.loc[i,'education'] == 'tertiary':
        df.loc[i,'education'] = 2
    elif df.loc[i,'education'] == 'secondary':
        df.loc[i,'education'] = 1
    else:
        df.loc[i,'education'] = 0

In [9]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,previous,poutcome,y
0,58,1,2,2,0,2143,1,0,0,261,1,0,0,0
1,44,0,1,1,0,29,1,0,0,151,1,0,0,0
2,33,0,2,1,0,2,1,1,0,76,1,0,0,0
3,35,1,2,2,0,231,1,0,0,139,1,0,0,0
4,28,1,1,2,0,447,1,1,0,217,1,0,0,0


In [10]:
df.job = df.job.astype('int64')
df.education = df.education.astype('int64')
df.marital = df.marital.astype('int64')

In [11]:
df.describe().round(2)

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,previous,poutcome,y
count,34727.00,34727.00,34727.00,34727.00,34727.00,34727.00,34727.00,34727.00,34727.00,34727.00,34727.00,34727.0,34727.00,34727.00
mean,40.42,0.21,1.49,1.15,0.02,1323.33,0.57,0.17,0.08,184.93,2.14,0.6,0.19,0.07
std,9.92,0.41,0.69,0.66,0.13,3004.05,0.50,0.37,0.28,110.85,1.33,2.4,0.39,0.26
min,18.00,0.00,0.00,0.00,0.00,-8019.00,0.00,0.00,0.00,0.00,1.00,0.0,0.00,0.00
25%,32.00,0.00,1.00,1.00,0.00,69.00,0.00,0.00,0.00,99.00,1.00,0.0,0.00,0.00
50%,39.00,0.00,2.00,1.00,0.00,431.00,1.00,0.00,0.00,162.00,2.00,0.0,0.00,0.00
75%,48.00,0.00,2.00,2.00,0.00,1378.00,1.00,0.00,0.00,254.00,3.00,0.0,0.00,0.00
max,70.00,1.00,2.00,2.00,1.00,102127.00,1.00,1.00,1.00,480.00,6.00,275.0,1.00,1.00


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34727 entries, 0 to 34726
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   age        34727 non-null  int64
 1   job        34727 non-null  int64
 2   marital    34727 non-null  int64
 3   education  34727 non-null  int64
 4   default    34727 non-null  int64
 5   balance    34727 non-null  int64
 6   housing    34727 non-null  int64
 7   loan       34727 non-null  int64
 8   contact    34727 non-null  int64
 9   duration   34727 non-null  int64
 10  campaign   34727 non-null  int64
 11  previous   34727 non-null  int64
 12  poutcome   34727 non-null  int64
 13  y          34727 non-null  int64
dtypes: int64(14)
memory usage: 3.7 MB


**2 - model training**

In [13]:
from sklearn.model_selection import train_test_split
import numpy as np

In [14]:
X = df.iloc[:, :13]
y = df.iloc[:,13]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                            test_size=0.3,
                            random_state=2)

In [15]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

model = LogisticRegression(penalty='l2')
model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 7.5 %
Out-of-sample Classification Error: 7.32 %


/Applications/python-anaconda/install/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=50, max_depth=7)

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 6.3 %
Out-of-sample Classification Error: 6.36 %


In [17]:
# LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis()

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 7.39 %
Out-of-sample Classification Error: 7.35 %


In [18]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(criterion='entropy',max_depth=4)

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 6.29 %
Out-of-sample Classification Error: 6.370000000000001 %


In [19]:
# Boosting methods
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(learning_rate=0.65,n_estimators=100)

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 6.81 %
Out-of-sample Classification Error: 6.79 %


In [20]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=150) 

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 5.87 %
Out-of-sample Classification Error: 6.35 %


In [21]:
# Neural network
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(max_iter=2000)

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 7.33 %
Out-of-sample Classification Error: 7.12 %


In [22]:
# feature selection

In [23]:
from scipy.stats import pearsonr
for i in range(len(df.columns)):
    print(df.columns[i],':',pearsonr(df.iloc[:,i],df['y'])[0])

age : -0.006262441065390461
job : 0.04557929609642366
marital : -0.03029426315150697
education : 0.09730470557888482
default : -0.030768012786176177
balance : 0.051353840255327174
housing : -0.17092331628218338
loan : -0.07657120440813676
contact : -0.013408101760176129
duration : 0.20868722769736742
campaign : -0.0817132096821789
previous : 0.11239577683159696
poutcome : 0.12530884850272467
y : 0.9999999999999768


In [24]:
df_sub_1 = df[['job','education','balance','housing','loan','duration','campaign','previous','poutcome','y']]

In [25]:
X = df_sub_1.iloc[:, :9]
y = df_sub_1.iloc[:,9]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                            test_size=0.3,
                            random_state=2)

In [26]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=50, max_depth=7)

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 6.17 %
Out-of-sample Classification Error: 6.3 %


In [27]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(criterion='entropy',max_depth=4)

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 6.34 %
Out-of-sample Classification Error: 6.239999999999999 %


In [28]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=150) 

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 6.04 %
Out-of-sample Classification Error: 6.39 %


In [29]:
df_sub_2 = df[['housing','duration','campaign','previous','poutcome','y']]

In [30]:
X = df_sub_2.iloc[:, :5]
y = df_sub_2.iloc[:,5]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                            test_size=0.3,
                            random_state=2)

In [31]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=50, max_depth=7)

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 6.18 %
Out-of-sample Classification Error: 6.35 %


In [32]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(criterion='entropy',max_depth=4)

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 6.34 %
Out-of-sample Classification Error: 6.239999999999999 %


In [33]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=150) 

model.fit(X_train, y_train)

tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
print('In-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

tn, fp, fn, tp = confusion_matrix(y_test, model.predict(X_test)).ravel()
print('Out-of-sample Classification Error:', round((fp+fn)/(tn+fp+fn+tp),4)*100, '%')

In-sample Classification Error: 6.1899999999999995 %
Out-of-sample Classification Error: 6.279999999999999 %
